## Building model

In [107]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Định nghĩa mạng CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)  # Grayscale: 1 kênh
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)  # MNIST ảnh 28x28 -> sau 2 lần pooling: (28/2/2) = 7
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)  # 10 lớp đầu ra cho MNIST (chữ số 0-9)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten tensor trước khi đưa vào Fully Connected
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

## Preparing dataset

In [110]:
# Tải dữ liệu MNIST
transform = transforms.Compose([
    transforms.ToTensor(),  # Chuyển ảnh thành tensor
    transforms.Normalize((0.5,), (0.5,))  # Chuẩn hóa giá trị pixel về [-1, 1]
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


## Defining loss function and optimizer

In [100]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()  # Hàm mất mát cho bài toán phân loại
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Bộ tối ưu hóa Adam

## Training model

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Chế độ huấn luyện
    running_loss = 0.0
    for idx, (inputs, labels) in enumerate(tqdm(train_loader)):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
model.eval()  # Chế độ đánh giá
correct = 0
total = 0

with torch.no_grad():
    for idx, (inputs, labels) in enumerate(tqdm(test_loader)):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")